In [1]:
import weave

In [11]:
import weave
from api2.engine import init_engine
client = weave.init("shawn/programmer-weave-analytics-pipeline")
init_engine(client)
call = client.call("affbd204-d54e-4c14-92e3-a3e6683895c1")

weave version 0.50.13 is available!  To upgrade, please run:
 $ pip install weave --upgrade
Logged in as Weights & Biases user: shawn.
View Weave data at https://wandb.ai/shawn/programmer-weave-analytics-pipeline/weave


In [6]:
steps = list(call.children())

In [8]:
last_step = steps[-1]

In [10]:
last_step.inputs['state']

TraceObject(ObjectRecord({'name': None, 'description': None, 'history': [{'role': 'user', 'content': 'can you modify get_user_calls_by_hour.py to include sums of whatever key paths are present in the summary_dump (json string) field of the calls_merged table? you should use tool_clickhouse_query to inspect some example data'}, {'role': 'assistant', 'tool_calls': [{'id': 'call_9M5HekOeakJaqKeL3zCAE0fj', 'function': {'arguments': '{"command":"cat get_user_calls_by_hour.py"}', 'name': 'run_command'}, 'type': 'function'}]}, {'tool_call_id': 'call_9M5HekOeakJaqKeL3zCAE0fj', 'role': 'tool', 'content': 'Exit code: 0\nSTDOUT\n# Get user calls by hour, email, op_name, with bytes used information and unique trace count\n\nfrom tool_clickhouse_query import tool_clickhouse_query\nfrom tool_wandb_production_query import tool_wandb_production_query\nimport base64\nimport pandas as pd\n\n\ndef get_user_calls_by_hour(limited_mode=True):\n    # Query to get user IDs, their emails, op_names, and count o

In [16]:
l = [dict(d) for d in list(last_step.inputs['state'].history)]

In [17]:
l

[{'role': 'user',
  'content': 'can you modify get_user_calls_by_hour.py to include sums of whatever key paths are present in the summary_dump (json string) field of the calls_merged table? you should use tool_clickhouse_query to inspect some example data'},
 {'role': 'assistant',
  'tool_calls': TraceList([{'id': 'call_9M5HekOeakJaqKeL3zCAE0fj', 'function': {'arguments': '{"command":"cat get_user_calls_by_hour.py"}', 'name': 'run_command'}, 'type': 'function'}])},
 {'tool_call_id': 'call_9M5HekOeakJaqKeL3zCAE0fj',
  'role': 'tool',
  'content': 'Exit code: 0\nSTDOUT\n# Get user calls by hour, email, op_name, with bytes used information and unique trace count\n\nfrom tool_clickhouse_query import tool_clickhouse_query\nfrom tool_wandb_production_query import tool_wandb_production_query\nimport base64\nimport pandas as pd\n\n\ndef get_user_calls_by_hour(limited_mode=True):\n    # Query to get user IDs, their emails, op_names, and count of calls created by hour from ClickHouse\n    clickh

In [41]:
import openai
import textwrap

# What do we want to know?
# we want a list of tasks, in order
# each task has
#   what was the user task
#   was the task successfully completed?
#   a list of descriptions of each error the model made
#   a list of additional user guidances beyond the initial task definition

@weave.op()
def summarize_run(id):
    call = client.call(id)
    steps = list(call.children())
    last_step = steps[-1]
    state = last_step.inputs['state']
    history = [dict(d) for d in list(state.history)]
    
    prompt = textwrap.dedent(f"""
    You will be provided with a conversation between a user and an LLM agent that has access to the local filesystem.

    <conversation>
    {history}
    </conversation>

    Define a "task" as a sequence of messages that starts with the user providing a goal, and ends with the user either changing the task
    (either by modifying the definition of the current task, or switching to an entirely new task), or the end of the conversation.

    For each task in the conversation, please provide the following:
    - what was the user task?
    - why did the task terminate? [update, replace, conversation_end] + rationale
    - task success score: [fail, success, partial_credit] + rationale
    - a list of short summaries of each error the model made
    - a list of additional user guidances beyond the initial task definition

    """)
    response = openai.chat.completions.create(
        model="gpt-4o", messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [43]:
result = summarize_run("affbd204-d54e-4c14-92e3-a3e6683895c1")
print(result)

🍩 https://wandb.ai/shawn/programmer-weave-analytics-pipeline/r/call/06f0d455-3b86-4444-8cbc-f50cafcdf1b0
### Task 1
#### User Task:
The user asked the assistant to modify the script `get_user_calls_by_hour.py` to include sums of the key paths present in the `summary_dump` JSON string field from the `calls_merged` table. The user also guided the assistant to use `tool_clickhouse_query` to inspect example data.

#### Task Termination: 
Update - The user requested confirmation of successful completion by running the script.

#### Task Success Score: 
Partial Credit - 
- **Rationale**: The assistant successfully modified the script to include the desired functionality and handled several exceptions during execution. The task included full implementation and final verification. However, there were errors during the script's execution that required handling multiple times, demonstrating a need for iterative debugging.

#### Error Summaries:
1. **Initial UnicodeDecodeError**: The assistant en

In [83]:
# OK this is a great example. How do we do this in batch efficiently?
# Who would it work in Weave1. It wouldn't. Unless this was a Weave op, with just the LLM part summarized.

# If I want to do this in batch

# OK so important things here:
#   - need to be able to do the same batch operations on call that I can do on a single one!
#     (so we can easily upgrade pipelines from one to many)
#   - this is why I went down the DBOp path in the first place, now that I remember.

runs = client.calls()

# TODO: this block is all batch. We want it to be the same API as on the single one.
run_steps = runs.children()
last_run_steps = run_steps[-1]
last_run_step_state_refs = last_run_steps.inputs['state']
last_run_step_states = last_run_step_state_refs.get()
last_run_step_state_histories = last_run_step_states.attr('history')

last_run_step_state_history_vals = last_run_step_state_histories.to_pandas()

called = summarize_run_rollout_llm.batch_call(last_run_step_state_history_vals)
called.cost()
called.execute()


@weave.op()
def summarize_run_rollout(id):
    # Data fetching logic
    call = client.call(id)
    steps = list(call.children())
    last_step = steps[-1]
    state = last_step.inputs['state']
    history = [dict(d) for d in list(state.history)]
    history_s = str(history)

    # LLM logic
    history_truncated_s = ''
    if len(history_s) > 150 * 1024:
        history_s = history_s[:150 * 1024] + '...'
        history_truncated_s = 'HISTORY TRUNCATED\n'
    
    prompt = textwrap.dedent(f"""
    You will be provided with a conversation between a user and an LLM agent that has access to the local filesystem.

    <conversation>
    {history_s}
    </conversation>
    {history_truncated_s}

    The conversation proceeds by the user providing input (a "user step") followed by N "assistant steps" where the assistant does work.

    The first user input in a conversation defines a Task, which has a Goal.

    Each user step we encounter may do one of the following actions:
    - Update Goal for current Task
    - Replace Goal starting new Task
    - Provide Guidance for current Task starting SubTask

    The Task and SubTask cannot change unless the user intervenes!

    Please provide summary:
    - User Task: <task definition summary>
      - User SubTask: <subtask definition summary>
        - Assistant: <assistant step summary>
        - Assistant: <assistant step summary>
        - ...
      - ...
      - Success: [true|false|partial]
    - ...
    """)
    response = openai.chat.completions.create(
        model="gpt-4o", messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [84]:
summary = summarize_run_rollout("affbd204-d54e-4c14-92e3-a3e6683895c1")
print(summary)

🍩 https://wandb.ai/shawn/programmer-weave-analytics-pipeline/r/call/29893adc-ff30-41e2-aed6-1e8ef2d28293
- User Task: Modify `get_user_calls_by_hour.py` to include sums of key paths from the `summary_dump` field of the `calls_merged` table.
  - User SubTask: Inspect example data from `calls_merged` table's `summary_dump` field.
    - Assistant: Inspected the content of `get_user_calls_by_hour.py` using `tool_clickhouse_query`.
    - Assistant: Inspected the content of `tool_clickhouse_query.py` using `tool_clickhouse_query`.
    - Assistant: Ran a query to inspect `summary_dump` data from `calls_merged` using `tool_clickhouse_query.py`.
    - Assistant: Processed and included an example of `summary_dump` content in the response.
  - Success: true
  - User SubTask: Update `get_user_calls_by_hour.py` script to aggregate key paths from `summary_dump`.
    - Assistant: Updated `get_user_calls_by_hour.py` script to include parsing and summing of key paths in `summary_dump`.
    - Assistant:

In [85]:
summary = summarize_run_rollout("e5ea1118-d045-4c0b-b4e8-cf85a21349ca")
print(summary)

🍩 https://wandb.ai/shawn/programmer-weave-analytics-pipeline/r/call/f6837270-762d-47bf-b1aa-bc97c826639f
- User Task: Update `plot_weekly_retention.py` to improve the output
  - User SubTask: Check the initial output of `pipeline_weekly_retention.py` by running it and viewing the plot.
    - Assistant: Listed files in the directory.
    - Assistant: Read the content of `pipeline_weekly_retention.py`.
    - Assistant: Read the content of `plot_weekly_retention.py`.
    - Assistant: Ran the `pipeline_weekly_retention.py` to generate initial output.
    - Assistant: Viewed the generated image (`retention_plot.png`).
    - Success: true
  - User SubTask: Modify `plot_weekly_retention.py` to filter out the first week, add numbers to each cell, and remove the background grid.
    - Assistant: Updated the `plot_weekly_retention.py` script with the requested changes.
    - Assistant: Ran the `pipeline_weekly_retention.py` to generate modified output.
    - Assistant: Viewed the newly generated

In [ ]:
# I want a nice LLM conversation browser that I can annotate with LLMs.
# Maybe I write an op that given a conversation, builds a parallel annotation structure with the info I want.

In [3]:
from api2.provider import calls

In [39]:
q = calls(client, 'run', limit=10)
q = q.children()
q = q.nth(-1)
q = q.column('inputs.state')
q = q.expand_ref()

In [34]:
df = q.to_pandas()

In [40]:
df

parent_ref
weave:///shawn/programmer-weave-analytics-pipeline/call/ec28d3bc-d690-4681-b7b5-1925b46f3c78    weave:///shawn/programmer-weave-analytics-pipe...
weave:///shawn/programmer-weave-analytics-pipeline/call/4a8bbddd-d63f-4e07-b459-986bfe7b18df    weave:///shawn/programmer-weave-analytics-pipe...
weave:///shawn/programmer-weave-analytics-pipeline/call/295005f0-6173-408f-bf09-4efe2c4c38f9    weave:///shawn/programmer-weave-analytics-pipe...
weave:///shawn/programmer-weave-analytics-pipeline/call/e5ea1118-d045-4c0b-b4e8-cf85a21349ca    weave:///shawn/programmer-weave-analytics-pipe...
weave:///shawn/programmer-weave-analytics-pipeline/call/32ff68d3-5e05-4b90-978d-caba02a45442    weave:///shawn/programmer-weave-analytics-pipe...
weave:///shawn/programmer-weave-analytics-pipeline/call/01f8251f-13e9-4717-8156-b262411b8cfe    weave:///shawn/programmer-weave-analytics-pipe...
weave:///shawn/programmer-weave-analytics-pipeline/call/e3ef453c-834b-4b92-b462-43a295e47be2    weave:///shawn/pr